In [1]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer, colors

In [2]:
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

In [3]:
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [4]:
viz.display(robot.q0)

In [80]:
q0 = robot.q0
v0 = np.zeros(robot.nv)
a0 = np.zeros(robot.nv)
x0 = np.concatenate([q0, v0])

In [243]:
nq = cmodel.nq
nv = cmodel.nv
nx = nq + nv
ndx = nv *2

cx = casadi.SX.sym('cx', nx, 1)
cdx = casadi.SX.sym('cx', ndx, 1)
ca = casadi.SX.sym('ca', nv, 1)
xref = casadi.SX.sym('cx', nx, 1)

IDX_LF = cmodel.getFrameId('leg_left_6_link')
IDX_RF = cmodel.getFrameId('leg_right_6_link')
IDX_LG = cmodel.getFrameId('gripper_left_base_link')
IDX_RG = cmodel.getFrameId('gripper_right_base_link')

integrate = casadi.Function('integrate', [cx, cdx], [ casadi.vertcat(cpin.integrate(cmodel,cx[:nq],cdx[:nv]),
                                                        cx[-nv:]+cdx[-nv:]) ] )
                                                        
com_position = casadi.Function('com', [cx], [cpin.centerOfMass(cmodel, cdata, cx[: nq])] )

cpin.forwardKinematics(cmodel, cdata, cx[: nq], cx[nq :], ca)
cpin.updateFramePlacements(cmodel, cdata)

lf_position = casadi.Function('lf_pos', [cx], [cdata.oMf[IDX_LF].translation])
lf_rotation = casadi.Function('lf_rot', [cx], [cdata.oMf[IDX_LF].rotation])
rf_position = casadi.Function('rf_pos', [cx], [cdata.oMf[IDX_RF].translation])
rf_rotation = casadi.Function('rf_rot', [cx], [cdata.oMf[IDX_RF].rotation])

lg_position = casadi.Function('lg_pos', [cx], [cdata.oMf[IDX_LG].translation])
lg_rotation = casadi.Function('lg_rot', [cx], [cdata.oMf[IDX_LG].rotation])
rg_position = casadi.Function('rg_pos', [cx], [cdata.oMf[IDX_RG].translation])
rg_rotation = casadi.Function('rg_rot', [cx], [cdata.oMf[IDX_RG].rotation])


# Tried with log method, which probably is more robust, but I was not able to make it work
#get_T_rf = casadi.Function('T_rf', [cx], [cpin.log6(cpin.SE3(rf_rotation(cx), rf_position(cx) ) ).vector])


# arm_distance = casadi.Function('arm_dist', [cx], [cpin.log6(cpin.SE3(lg_rotation(cx), lg_position(cx) ).inverse() *
 #                                                       cpin.SE3(rg_rotation(cx), rg_position(cx) )).vector])


In [244]:
opti = casadi.Opti()

dxs = opti.variable(ndx)
xs = integrate(x0, dxs)

cost = casadi.sumsqr(xs - x0) + casadi.sumsqr(arm_distance(xs))
#casadi.sumsqr(lg_position(xs) - rg_position(xs) - 0.2)

opti.minimize(cost)

# COM
opti.subject_to(opti.bounded(-0.08, com_position(xs)[0], 0.08))
opti.subject_to(opti.bounded(-0.02, com_position(xs)[1], 0.02))
opti.subject_to(opti.bounded(0.7, com_position(xs)[2], 0.9))

# Standing foot
opti.subject_to(rf_position(xs) - rf_position(x0) == 0)
opti.subject_to(rf_rotation(xs) - rf_rotation(x0) == 0)

# Free foot
opti.subject_to(lf_position(xs)[2] >= 0.4)
opti.subject_to(opti.bounded(0.05, lf_position(xs)[0:2], 0.1))

r_ref = pin.utils.rotate('z', 3.14 / 2) @ pin.utils.rotate('y', 3.14 / 2) # orientation target
opti.subject_to(opti.bounded(-0.0, lf_rotation(xs) - r_ref, 0.0))


In [245]:
opti.solver('ipopt')
opti.set_initial(dxs, np.zeros(ndx))
opti.solve()

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      186
Number of nonzeros in inequality constraint Jacobian.:      144
Number of nonzeros in Lagrangian Hessian.............:      408

Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 1
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 1
Number of Lagrangian Hessian evaluations             = 0
Total CPU secs in IPOPT (w/o function evaluations)   =      0.001
Total CPU secs in NLP function evaluations     

CasADi - 2022-03-23 15:25:56 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 3, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2022-03-23 15:25:56 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 3, col 0).") [.../casadi/core/oracle_function.cpp:265]


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:159:
.../casadi/core/optistack_internal.cpp:997: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Number_Detected'

In [232]:
x_sol = integrate(x0, opti.value(dxs)).full()
q_sol = x_sol[: nq]
v_sol = x_sol[nq:]

In [233]:
viz.display(q_sol)